In [ ]:
import pandas as pd
import numpy as np
from glob import glob
import os
import itertools

import nltk
nltk.download("stopwords")
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
from nltk.corpus import stopwords
stopwords = stopwords.words('english')
stopwords += [".", ",", "?", "!", ":", "(", ")"]

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


## Reading

In [ ]:
def read_file(filename):
    with open(filename, "r") as file_:
        return file_.read()

def tokenize_to_sentence(text):
    return [sent for sent in nltk.sent_tokenize(text)]

def tokenize_to_words(sent):
    return [word for word in nltk.word_tokenize(sent) if word not in stopwords]

root_folder = "/content/drive/MyDrive/olya/texts"
engs = sorted(glob(os.path.join(root_folder, "eng/*.txt")))
ukr = sorted(glob(os.path.join(root_folder, "ukr/*.txt")))

eng_text = [read_file(text) for text in engs]
ukr_text = [read_file(text) for text in ukr]

eng_sentences = [tokenize_to_sentence(text) for text in eng_text]
ukr_sentences = [tokenize_to_sentence(text) for text in ukr_text]

eng_words = [list(itertools.chain(*[tokenize_to_words(sent)  for sent in text])) for text in eng_sentences]
ukr_words = [list(itertools.chain(*[tokenize_to_words(sent) for sent in text])) for text in ukr_sentences]

## Coeficients

In [ ]:
texts = [text for text in ["12 правил життя", "Ніколи не йдіть на компроміс", "9.99: Міф про чесну ціну", "Успішні виступи на TED", "Чорний лебідь", "Найбагатший чоловік у Вавилоні", "Витончене мистецтво забивати на все", "Чому нації занепадають"] for _ in range(2) ]#[f"text_{inx}" for inx in range(8) for _ in range(2) ]
versions = ["english", "ukrainian"] * 8
index = pd.MultiIndex.from_tuples(list(zip(texts, versions)), names=["text name", "language"])

metrics = ["Коефіцієнт лексичної різноманітності", "Коефіцієнт синтаксичної складності", "Індекс винятковості", "Індекс концентрації тексту"]
df = pd.DataFrame(columns=metrics, index=index)

In [ ]:
df.head()

Коефіцієнт лексичної різноманітності  \
text name                    language                                         
12 правил життя              english                                    NaN   
                             ukrainian                                  NaN   
Ніколи не йдіть на компроміс english                                    NaN   
                             ukrainian                                  NaN   
9.99: Міф про чесну ціну     english                                    NaN   

                                       Коефіцієнт синтаксичної складності  \
text name                    language                                       
12 правил життя              english                                  NaN   
                             ukrainian                                NaN   
Ніколи не йдіть на компроміс english                                  NaN   
                             ukrainian                                NaN   
9.99: Міф про чесну ціну     english                                  NaN   

                                       Індекс винятковості  \
text name                    language                        
12 правил життя              english                   NaN   
                             ukrainian                 NaN   
Ніколи не йдіть на компроміс english                   NaN   
                             ukrainian                 NaN   
9.99: Міф про чесну ціну     english                   NaN   

                                       Індекс концентрації тексту  
text name                    language                              
12 правил життя              english                          NaN  
                             ukrainian                        NaN  
Ніколи не йдіть на компроміс english                          NaN  
                             ukrainian                        NaN  
9.99: Міф про чесну ціну     english                          NaN

In [ ]:
def lexical_diversity(text):
    return len(set(text)) / len(text)

df.loc[df.index.isin(["english"], level=1), "Коефіцієнт лексичної різноманітності"] = [lexical_diversity(text) for text in eng_words]
df.loc[df.index.isin(["ukrainian"], level=1), "Коефіцієнт лексичної різноманітності"] = [lexical_diversity(text) for text in ukr_words]

In [ ]:
def syntax_complexity(text, sentences):
    return 1 - (sentences / len(set(text)))

df.loc[df.index.isin(["english"], level=1), "Коефіцієнт синтаксичної складності"] = [syntax_complexity(text, len(eng_sentences[inx])) for inx, text in enumerate(eng_words)]
df.loc[df.index.isin(["ukrainian"], level=1), "Коефіцієнт синтаксичної складності"] = [syntax_complexity(text, len(ukr_sentences[inx])) for inx, text in enumerate(ukr_words)]

In [ ]:
def n_times_words(text, times, strict=True): #повертає список слів, які є в тексті н-разів
    dct = dict(zip(*np.unique(text,return_counts=True)))
    n = 0
    for key, val in dct.items():
        if strict:
          if val == times:
              n += 1
        else:
          if val >= times:
              n += 1
    return n / len(text)

df.loc[df.index.isin(["english"], level=1), "Індекс винятковості"] = [n_times_words(text, 1) for text in eng_words]
df.loc[df.index.isin(["ukrainian"], level=1), "Індекс винятковості"] = [n_times_words(text, 1) for text in ukr_words]
df.loc[df.index.isin(["english"], level=1), "Індекс концентрації тексту"] = [n_times_words(text, 10, False) for text in eng_words]
df.loc[df.index.isin(["ukrainian"], level=1), "Індекс концентрації тексту"] = [n_times_words(text, 10, False) for text in ukr_words]

In [ ]:
df

Коефіцієнт лексичної різноманітності  \
text name                           language                                         
12 правил життя                     english                               0.184101   
                                    ukrainian                             0.215784   
Ніколи не йдіть на компроміс        english                               0.184248   
                                    ukrainian                             0.237797   
9.99: Міф про чесну ціну            english                               0.203175   
                                    ukrainian                             0.231884   
Успішні виступи на TED              english                               0.198201   
                                    ukrainian                             0.250751   
Чорний лебідь                       english                               0.170876   
                                    ukrainian                             0.227721   
Найбагатший чоловік у Вавилоні      english                               0.213164   
                                    ukrainian                             0.258696   
Витончене мистецтво забивати на все english                               0.215534   
                                    ukrainian                             0.249604   
Чому нації занепадають              english                               0.142854   
                                    ukrainian                              0.18869   

                                              Коефіцієнт синтаксичної складності  \
text name                           language                                       
12 правил життя                     english                             0.411033   
                                    ukrainian                           0.589405   
Ніколи не йдіть на компроміс        english                             0.543969   
                                    ukrainian                           0.696734   
9.99: Міф про чесну ціну            english                             0.539626   
                                    ukrainian                            0.67875   
Успішні виступи на TED              english                             0.456624   
                                    ukrainian                           0.673859   
Чорний лебідь                       english                             0.547595   
                                    ukrainian                           0.738395   
Найбагатший чоловік у Вавилоні      english                             0.544264   
                                    ukrainian                           0.685308   
Витончене мистецтво забивати на все english                              0.53349   
                                    ukrainian                           0.701401   
Чому нації занепадають              english                             0.476376   
                                    ukrainian                           0.682015   

                                              Індекс винятковості  \
text name                           language                        
12 правил життя                     english              0.098842   
                                    ukrainian            0.133576   
Ніколи не йдіть на компроміс        english              0.093018   
                                    ukrainian            0.147848   
9.99: Міф про чесну ціну            english                0.1105   
                                    ukrainian            0.140846   
Успішні виступи на TED              english              0.105385   
                                    ukrainian            0.156753   
Чорний лебідь                       english               0.08659   
                                    ukrainian             0.14011   
Найбагатший чоловік у Вавилоні      english              0.109352   
                                    ukrainian            0.163934   
Витончене мистецтв

In [ ]:
# !pip install stanza
import stanza
stanza.download('uk')
stanza.download("en")
nlp = stanza.Pipeline('uk')
nlp_en = stanza.Pipeline("en")

In [ ]:
docs = [nlp(text) for text in ukr_text]
docs_eng = [nlp_en(text) for text in eng_text]

In [ ]:
def tightness_coef(doc):
    sentences = doc.sentences
    prep = 0
    conj = 0
    for sent in doc.sentences:
        for word in sent.words:
            if word.upos == "IN":
                prep += 1
            elif word.upos == "CC":
                conj += 1
    return (prep + conj) / (3 * sentences)

df.loc[df.index.isin(["english"], level=1), "5"] = [tightness_coef(doc) for doc in docs_eng]
df.loc[df.index.isin(["ukrainian"], level=1), "5"] = [tightness_coef(doc) for doc in docs]

In [ ]:
texts = [f"text_{inx}" for inx in range(8) for _ in range(2) ]
versions = ["english", "ukrainian"] * 8
index = pd.MultiIndex.from_tuples(list(zip(texts, versions)), names=["text name", "language"])

metrics = ["nouns", "adjectives"]
pos_df = pd.DataFrame(columns=metrics, index=index)

In [ ]:
def get_pos_count(doc, pos):
    count = 0
    for sent in doc.sentences:
        for word in sent.words:
            if word.xpos in pos:
                count += 1
    return count

pos_df.loc[pos_df.index.isin(["english"], level=1), "nouns"] = [get_pos_count(doc, ["NNP", "NNS", "NN"]) for doc in docs_eng]
pos_df.loc[pos_df.index.isin(["english"], level=1), "adjectives"] = [get_pos_count(doc, "") for doc in docs_eng]
pos_df.loc[pos_df.index.isin(["ukrainian"], level=1), "nouns"] = [get_pos_count(doc, ["NNP", "NNS", "NN"]) for doc in docs]
pos_df.loc[pos_df.index.isin(["ukrainian"], level=1), "adjectives"] = [get_pos_count(doc, []) for doc in docs]

In [ ]:
def syllables(word):
    count = 0
    vowels = 'aeiouy'
    word = word.lower()
    if word[0] in vowels:
        count +=1
    for index in range(1,len(word)):
        if word[index] in vowels and word[index-1] not in vowels:
            count +=1
    if word.endswith('e'):
        count -= 1
    if word.endswith('le'):
        count += 1
    if count == 0:
        count += 1
    return count

def syllables_ukr(word):
    count = 0
    vowels = 'аеїиіуо'
    word = word.lower()
    for let in word:
        if let in vowels:
            count += 1
    return count
texts = [text for text in ["12 правил життя", "Ніколи не йдіть на компроміс", 
                           "9.99: Міф про чесну ціну", "Успішні виступи на TED", 
                           "Чорний лебідь", "Найбагатший чоловік у Вавилоні", 
                           "Витончене мистецтво забивати на все", 
                           "Чому нації занепадають"] for _ in range(2) ]
versions = ["english", "ukrainian"] * 8
index = pd.MultiIndex.from_tuples(list(zip(texts, versions)), names=["text name", "language"])

metrics = ["readability"]
read_df = pd.DataFrame(columns=metrics, index=index)

lst = []
for inx in range(len(ukr_text)):
    total_sent = len(ukr_sentences[inx])
    total_words = len(ukr_words[inx])
    total_syllables = np.sum([syllables_ukr(word) for word in ukr_words[inx]])
    lst.append(206.835 - 1.52 * (total_words/total_sent) - 65.15 * (total_syllables / total_words))

lst_en = []
for inx in range(len(eng_text)):
    total_sent = len(eng_sentences[inx])
    total_words = len(eng_words[inx])
    total_syllables = np.sum([syllables(word) for word in eng_words[inx]])
    lst_en.append(206.835 - 1.015 * (total_words/total_sent) - 84.6 * (total_syllables / total_words))

read_df.loc[read_df.index.isin(["english"], level=1), "readability"] = lst_en
read_df.loc[read_df.index.isin(["ukrainian"], level=1), "readability"] = lst

In [ ]:
read_df

readability
text name                           language             
12 правил життя                     english      30.48546
                                    ukrainian   59.689788
Ніколи не йдіть на компроміс        english     43.989011
                                    ukrainian   52.657116
9.99: Міф про чесну ціну            english     38.322464
                                    ukrainian   53.253849
Успішні виступи на TED              english      44.66997
                                    ukrainian   57.017791
Чорний лебідь                       english     20.457177
                                    ukrainian   45.679047
Найбагатший чоловік у Вавилоні      english     59.472653
                                    ukrainian   65.918825
Витончене мистецтво забивати на все english     42.078393
                                    ukrainian   59.871836
Чому нації занепадають              english      6.797776
                                    ukrainian   33.511944